In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist # loading the fashion MNIST dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

(X_full, y_full), (X_test_full, y_test_full) = fashion_mnist.load_data()

# Normalizing the image pixel values
X_full = X_full / 255.0
X_test_full = X_test_full / 255.0

# Reshape the data for the models
X_flat = X_full.reshape((X_full.shape[0], -1))
X_test_flat = X_test_full.reshape((X_test_full.shape[0], -1))

X_cnn = X_full.reshape((X_full.shape[0], 28, 28, 1))
X_test_cnn = X_test_full.reshape((X_test_full.shape[0], 28, 28, 1))

# Split data into two sets, training(70%) and testing(30%) sets
X_train_cnn, X_test_cnn, X_train_flat, X_test_flat, y_train, y_test = train_test_split(
    X_cnn, X_flat, y_full, test_size=0.3, random_state=42, stratify=y_full
)

# Data augmentation is done to improve CNN generalization
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10, # rotating the images by 10 degrees
    zoom_range=0.1, # zoom images by 10%
    width_shift_range=0.1, # Shifting images horizontally by 10%
    height_shift_range=0.1 #  Shifting images vertically by 10%
)
datagen.fit(X_train_cnn)

In [2]:
# SVM 
from sklearn.svm import SVC

# Create and training the model
svm_model = SVC(kernel='rbf', probability=True)  # probability=True is needed for soft predictions
svm_model.fit(X_train_flat, y_train)

SVC(probability=True)

In [3]:
#Random Forest classifier
from sklearn.ensemble import RandomForestClassifier

# Creating and training the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_flat, y_train)

RandomForestClassifier(random_state=42)

In [4]:
#Imported to build the CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import Input

#Building and training the model
cnn_model = Sequential([
    Input(shape=(28, 28, 1)), 
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Dropout for regularization
    Dense(10, activation='softmax') # Output layer for 10 classes
])

# Compiling the model
cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Train the model
cnn_model.fit(X_train_cnn, y_train,
              batch_size=64,
              epochs=10,
              validation_data=(X_test_cnn, y_test))

Epoch 1/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 23s 31ms/step - accuracy: 0.6584 - loss: 0.9489 - val_accuracy: 0.8463 - val_loss: 0.4079
Epoch 2/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.8330 - loss: 0.4649 - val_accuracy: 0.8669 - val_loss: 0.3509
Epoch 3/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - accuracy: 0.8577 - loss: 0.3896 - val_accuracy: 0.8859 - val_loss: 0.3117
Epoch 4/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.8718 - loss: 0.3508 - val_accuracy: 0.8947 - val_loss: 0.2859
Epoch 5/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.8814 - loss: 0.3217 - val_accuracy: 0.8917 - val_loss: 0.2896
Epoch 6/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.8897 - loss: 0.2978 - val_accuracy: 0.9012 - val_loss: 0.2674
Epoch 7/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 22s 30ms/step - accuracy: 0.8964 - loss: 0.2879 - val_accuracy: 0.9035 - val_loss: 0.2614
Epoch 8/10
657/657 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.9012 - loss: 0.2735 - 

In [ ]:
# Individual modelPredictions are obtained
svm_preds = svm_model.predict(X_test_flat)
rf_preds = rf_model.predict(X_test_flat)
cnn_preds = np.argmax(cnn_model.predict(X_test_cnn), axis=1)

# Probability predictions for ensemble are obtained
svm_probs = svm_model.predict_proba(X_test_flat)
rf_probs = rf_model.predict_proba(X_test_flat)
cnn_probs = cnn_model.predict(X_test_cnn)

# Average the probabilities from all the three models i.e., "SVM, Random forest, CNN"
Average_probs = (svm_probs + rf_probs + cnn_probs) / 3
ensemble_preds = np.argmax(Average_probs, axis=1)

563/563 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step


In [ ]:
#import the evaluate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Defined the function to evaluate and print metrics for each model
def evaluate_model(y_true, y_pred, model_name):
    ACCURACY = accuracy_score(y_true, y_pred)
    PRECISION = precision_score(y_true, y_pred, average='macro')
    RECALL = recall_score(y_true, y_pred, average='macro')
    f1_SCORE = f1_score(y_true, y_pred, average='macro')
    
    print(f"\n{model_name} Evaluation:")
    print(f"Accuracy - {ACCURACY:.4f}")
    print(f"Precision - {PRECISION:.4f}")
    print(f"Recall - {RECALL:.4f}")
    print(f"F1 Score - {f1_SCORE:.4f}")
    
# Evaluate the models
evaluate_model(y_test, svm_preds, "SVM")
evaluate_model(y_test, rf_preds, "Random Forest")
evaluate_model(y_test, cnn_preds, "CNN")
evaluate_model(y_test, ensemble_preds, "Ensemble")